<a href="https://colab.research.google.com/github/chimaobi-okite/Tourism_classification/blob/main/tour_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Toursist Challenge Feature Engineering And Modelling

### setup

In [1]:
! pip install lightgbm
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Reading the data

In [2]:
## import the neccessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Evaluation metrics and validation
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

## Modelling
import lightgbm as lgb
import catboost as ctb
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/Test.csv')
var_def = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/VariableDefinitions.csv')
ss = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/SampleSubmission.csv')

### Data Preprocessing

In [4]:
## create a function that fills the missing values based on our eda insights

def fill_na(travel_with , main_col , other_col):
   if np.isnan(main_col):
     if travel_with == 'Alone' and other_col == 1.0: 
       return 0
     else:
        return 1
   else:
    return main_col

def fill_missing(data):
  df = data.copy()
  df['travel_with'] = df['travel_with'].fillna('Alone')
  df['total_male'] = df.apply(lambda row : fill_na(row['travel_with'], row['total_male'], row['total_female']), axis=1)
  df['total_female'] = df.apply(lambda row : fill_na(row['travel_with'], row['total_female'], row['total_male']), axis=1)
  return df

In [5]:
## function performs all preprocessing steps of the features in preparartion for modelling
def process_features(data):
  df = data.copy()
  ## fill missing values
  df = fill_missing(df)

  ## map package column to 1 and 0
  package_cols = ['package_transport_int', 'package_accomodation',
        'package_food', 'package_transport_tz', 'package_sightseeing',
        'package_guided_tour', 'package_insurance']
  for col in package_cols:
    df[col] = df[col].map(lambda x : 1 if x == 'Yes' else 0)

  ## generate a new column that counts how many packages a package tourist subscribes to
  df['package_count'] = df[package_cols].sum(axis = 1)
  df['total_nights'] = df['night_mainland'] + df['night_zanzibar']
  df['total_individuals'] = df['total_female'] + df['total_male']

  ## encode country
  country_nom = (train.groupby('country').size()) / len(train)
  country = train.country.value_counts().index
  df['country_code'] = df['country'].map(lambda x : country_nom[x] if x in country else 0.0)

  #drop 'Tour_id column and country
  df = df.drop(['Tour_ID', 'country'], axis =1)

  ## convert categorical columns to onehot values
  df = pd.get_dummies(df)

  ## rename columns to values acceptable by our models
  df = df.rename(columns= {'age_group_65+':'age_group_65_plus','age_group_<18':'age_group_18_less'})
  
  return df


In [6]:
## divide the dataset into features and target
df = train.copy()
X_, y = df.drop(['cost_category'], axis =1), df['cost_category']
## process features
X = process_features(X_)
## get label categories
targets = {'Lower Cost':1, 'Low Cost':2, 'Normal Cost':3, 'High Cost': 4,'Higher Cost': 5, 'Highest Cost' : 6}
y = y.map(targets)
## split data to train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Modelling

In [7]:
## function to validate data
def validate( estimator, X, y):
  cv = StratifiedKFold(n_splits=5,shuffle = True, random_state=1)
  scores = cross_val_score(estimator= estimator, X =X, y = y, cv = cv, scoring = 'neg_log_loss')
  return scores.mean(), scores.std(), scores

In [8]:
# inizialize the various models to be tested in this task
xgmodel = xgb.XGBClassifier(seed = 42, max_depth=5)
cat_model = ctb.CatBoostClassifier(random_seed = 42,max_depth=5)
lgb_model = lgb.LGBMClassifier(random_state=42, max_depth=5,num_leaves=25)
gra_model = GradientBoostingClassifier(random_state=42)
dmodel = DecisionTreeClassifier(random_state = 42, max_depth= 3)
rmodel = RandomForestClassifier(random_state =2,max_depth=11)
histmodel = HistGradientBoostingClassifier(random_state = 42,max_depth = 4)
extree = ExtraTreesClassifier(random_state = 42, max_depth = 12)

In [9]:
## cross validate across several models and show results
def validate_models():
  loss = []
  deviations = []
  test_loss = []

  names = ['xgmodel','cat_model','lgb_model','gra_model','dmodel', 'rmodel','histmodel', 'extree']
  models = [xgmodel,cat_model,lgb_model,gra_model,dmodel, rmodel,histmodel, extree]
  
  for model in models:
    mean_loss, std, _ = validate(model, X_train,y_train)
    model.fit(X_train,y_train)
    val_loss = log_loss(y_test, model.predict_proba(X_test))
    loss.append(mean_loss)
    deviations.append(std)
    test_loss.append(val_loss)
  
  summary = pd.DataFrame({'Model': names, 'Mean_Loss': loss, 'Std': deviations, 'Test_Loss': test_loss})
  return summary


In [10]:
validate_models()

Streaming output truncated to the last 5000 lines.
4:	learn: 1.4495617	total: 76.5ms	remaining: 15.2s
5:	learn: 1.4137432	total: 90.5ms	remaining: 15s
6:	learn: 1.3804388	total: 105ms	remaining: 14.9s
7:	learn: 1.3528983	total: 119ms	remaining: 14.8s
8:	learn: 1.3280779	total: 133ms	remaining: 14.6s
9:	learn: 1.3087693	total: 146ms	remaining: 14.4s
10:	learn: 1.2904359	total: 160ms	remaining: 14.4s
11:	learn: 1.2767228	total: 174ms	remaining: 14.3s
12:	learn: 1.2615018	total: 188ms	remaining: 14.3s
13:	learn: 1.2478012	total: 207ms	remaining: 14.6s
14:	learn: 1.2345263	total: 222ms	remaining: 14.6s
15:	learn: 1.2236711	total: 236ms	remaining: 14.5s
16:	learn: 1.2134128	total: 249ms	remaining: 14.4s
17:	learn: 1.2052658	total: 262ms	remaining: 14.3s
18:	learn: 1.1980328	total: 274ms	remaining: 14.2s
19:	learn: 1.1911861	total: 288ms	remaining: 14.1s
20:	learn: 1.1842493	total: 302ms	remaining: 14.1s
21:	learn: 1.1783811	total: 316ms	remaining: 14s
22:	learn: 1.1732926	total: 329ms	remai

,Model,Mean_Loss,Std,Test_Loss
0,xgmodel,-1.082588,0.019388,1.078350
1,cat_model,-1.090105,0.023251,1.080700
2,lgb_model,-1.081664,0.020115,1.078974
3,gra_model,-1.087808,0.019026,1.088547
4,dmodel,-1.206714,0.017211,1.224279
5,rmodel,-1.099389,0.017728,1.112719
6,histmodel,-1.085431,0.017819,1.084374
7,extree,-1.139064,0.014735,1.162077


#### Stacking Ensemble

In [11]:
estimators = [('xgb',xgmodel),('lgb',lgb_model), ('graident', gra_model),('cat_model', cat_model), ('tree', dmodel),('forest', rmodel)]
stc_ensemble = StackingClassifier(estimators,xgb.XGBClassifier(seed = 42))
stc_ensemble.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
4:	learn: 1.4521172	total: 72.4ms	remaining: 14.4s
5:	learn: 1.4164582	total: 87.7ms	remaining: 14.5s
6:	learn: 1.3836748	total: 102ms	remaining: 14.5s
7:	learn: 1.3564016	total: 117ms	remaining: 14.5s
8:	learn: 1.3330075	total: 131ms	remaining: 14.4s
9:	learn: 1.3136862	total: 144ms	remaining: 14.3s
10:	learn: 1.2945318	total: 159ms	remaining: 14.3s
11:	learn: 1.2784864	total: 173ms	remaining: 14.3s
12:	learn: 1.2635506	total: 187ms	remaining: 14.2s
13:	learn: 1.2503874	total: 210ms	remaining: 14.8s
14:	learn: 1.2372235	total: 224ms	remaining: 14.7s
15:	learn: 1.2259604	total: 238ms	remaining: 14.7s
16:	learn: 1.2160549	total: 259ms	remaining: 15s
17:	learn: 1.2074312	total: 271ms	remaining: 14.8s
18:	learn: 1.2003981	total: 284ms	remaining: 14.7s
19:	learn: 1.1939353	total: 298ms	remaining: 14.6s
20:	learn: 1.1879240	total: 311ms	remaining: 14.5s
21:	learn: 1.1818810	total: 324ms	remaining: 14.4s
22:	learn: 1.1750831	total: 339ms	rem

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(max_depth=5,
                                              objective='multi:softprob',
                                              seed=42)),
                               ('lgb',
                                LGBMClassifier(max_depth=5, num_leaves=25,
                                               random_state=42)),
                               ('graident',
                                GradientBoostingClassifier(random_state=42)),
                               ('cat_model',
                                <catboost.core.CatBoostClassifier object at 0x7f225812abd0>),
                               ('tree',
                                DecisionTreeClassifier(max_depth=3,
                                                       random_state=42)),
                               ('forest',
                                RandomForestClassifier(max_depth=11,
                                     

In [12]:
log_loss(y_test, stc_ensemble.predict_proba(X_test))

1.0720950281594668

### Making Submission

from the above it is obvious that the stack ensemble gave the best result. Lets use  it for submission

In [13]:
## fit tthat stack ensemble on the whole data
stc_ensemble.fit(X, y)

Streaming output truncated to the last 5000 lines.
4:	learn: 1.4547691	total: 87.6ms	remaining: 17.4s
5:	learn: 1.4194536	total: 106ms	remaining: 17.5s
6:	learn: 1.3865712	total: 124ms	remaining: 17.6s
7:	learn: 1.3586678	total: 141ms	remaining: 17.4s
8:	learn: 1.3344354	total: 158ms	remaining: 17.4s
9:	learn: 1.3121742	total: 175ms	remaining: 17.3s
10:	learn: 1.2933814	total: 191ms	remaining: 17.2s
11:	learn: 1.2778915	total: 218ms	remaining: 18s
12:	learn: 1.2625094	total: 235ms	remaining: 17.9s
13:	learn: 1.2494407	total: 258ms	remaining: 18.2s
14:	learn: 1.2375103	total: 275ms	remaining: 18.1s
15:	learn: 1.2265571	total: 294ms	remaining: 18.1s
16:	learn: 1.2163183	total: 312ms	remaining: 18.1s
17:	learn: 1.2079570	total: 329ms	remaining: 18s
18:	learn: 1.2005767	total: 346ms	remaining: 17.9s
19:	learn: 1.1937602	total: 363ms	remaining: 17.8s
20:	learn: 1.1868133	total: 380ms	remaining: 17.7s
21:	learn: 1.1817155	total: 395ms	remaining: 17.6s
22:	learn: 1.1765036	total: 411ms	remain

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(max_depth=5,
                                              objective='multi:softprob',
                                              seed=42)),
                               ('lgb',
                                LGBMClassifier(max_depth=5, num_leaves=25,
                                               random_state=42)),
                               ('graident',
                                GradientBoostingClassifier(random_state=42)),
                               ('cat_model',
                                <catboost.core.CatBoostClassifier object at 0x7f225812abd0>),
                               ('tree',
                                DecisionTreeClassifier(max_depth=3,
                                                       random_state=42)),
                               ('forest',
                                RandomForestClassifier(max_depth=11,
                                     

### Note

The models, hyperparameters, features etc used above didnt just come on the fly. I hand-tuned the hyperparameters of each model as performing GridSearch was too Expensive for my device. 

I also tried several feature engineering and tranformations that didnt help much like encoding regions and subregions, transforming some categorical columns to fewer categories based on EDA, droping some columns based on xgboost feature importance (This helped for xgboost but seemed removing those features affected other models in my stack thereby increasing the overall loss). 

In [14]:
## process test set
t = process_features(test)
## predict probability since the leaderboard uses log_loss metric
preds = stc_ensemble.predict_proba(t)

In [15]:
#create a submission file
sub_df = pd.DataFrame({'Tour_ID':test['Tour_ID'], 'High Cost': preds[:,3],'Higher Cost': preds[:,4],'Highest Cost': preds[:,5],'Low Cost': preds[:,1],'Lower Cost': preds[:,0],'Normal Cost' :preds[:,2]})
sub_df.to_csv('submission_main',index = False)

## Going Beyond Competition

The submission above performed well above my baseline and greatly improved by LeaderBoard Score. But in practice such model will be too costly to productionalize. Why not use a simple fine-tuned xgboost model with good/meaningful feature engineering?We know that the loss isnt that far off from the stacked model and we would be saving alot of computational resources, so its worth it.

In [16]:
model = xgb.XGBClassifier(seed=42, random_state = 42, max_depth = 5, average= 'weighted')
model.fit(X_train, y_train)
log_loss(y_test, model.predict_proba(X_test))

1.0783495608010787

### Model Interpretation

Getting Feature Importance

In [17]:
importance = model.feature_importances_
impo_df = pd.DataFrame({'Features': X_train.columns, 'Relevance': importance})
impo_df = impo_df.sort_values(by ='Relevance', ascending = False)
impo_df

,Features,Relevance
51,tour_arrangement_Independent,0.241570
26,purpose_Leisure and Holidays,0.120412
11,package_count,0.107874
20,travel_with_Alone,0.046248
13,total_individuals,0.031109
12,total_nights,0.024401
18,age_group_65_plus,0.024257
2,package_transport_int,0.022611
30,purpose_Scientific and Academic,0.018492
31,purpose_Visiting Friends and Relatives,0.014070


From the above we can see the most important features for cost category classification based on xgboost model is the tour_arrangement, followed by purpose. Our hand engineered features are great too. 

But how can we tell which columns are not so important atall? 
Well, create a random column, fit the model and check the feature importance, columns whose features are below the random model should not be of muuch use.

In [18]:
# copy X_train
X_random = X_train.copy()
# create a column with just random values
X_random['Random'] = np.random.rand(X_random.shape[0])
#fit
model.fit(X_random, y_train)
#check importance
importance = model.feature_importances_
impo_df = pd.DataFrame({'Features': X_random.columns, 'Relevance': importance})
impo_df = impo_df.sort_values(by ='Relevance', ascending = False)
impo_df


,Features,Relevance
51,tour_arrangement_Independent,0.244940
26,purpose_Leisure and Holidays,0.114631
11,package_count,0.106791
20,travel_with_Alone,0.044832
13,total_individuals,0.030901
12,total_nights,0.024612
18,age_group_65_plus,0.024365
2,package_transport_int,0.022104
30,purpose_Scientific and Academic,0.017622
31,purpose_Visiting Friends and Relatives,0.013931


The above shows the not so important features in our model and the not so important categories which canbe trimmed out. see function that does this below

In [19]:
def simplify_features(df):
  package_cols = ['package_transport_int', 'package_accomodation',
        'package_food', 'package_transport_tz', 'package_sightseeing',
        'package_guided_tour', 'package_insurance']

  for col in package_cols:
    df[col] = df[col].map(lambda x : 1 if x == 'Yes' else 0)
  df['package_count'] = df[package_cols].sum(axis = 1)
  df['random'] = np.random.rand(df.shape[0])
  df['total_nights'] = df['night_mainland'] + df['night_zanzibar']
  df['total_individuals'] = df['total_female'] + df['total_male']

  ## encode country

  country_nom = (train.groupby('country').size()) / len(train)
  country = train.country.value_counts().index
  df['country_code'] = df['country'].map(lambda x : country_nom[x] if x in country else 0.0)

  ## laabel encode tour arrangement column
  df['tour_arrangement'] = df['tour_arrangement'].map(lambda x : 1 if x == 'Independent' else 0)

  ## group the no so important categories in the columns below to one category
  df['main_activity'] = df['main_activity'].map(lambda x : group_features(x, 'activity'))
  df['info_source'] = df['info_source'].map(lambda x : group_features(x, 'info'))
  df['travel_with'] = df['travel_with'].map(lambda x : group_features(x, 'travel_with'))
  df['purpose'] = df['purpose'].map(lambda x : group_features(x, 'purpose'))

  # drop the not needed columns
  df = df.drop(['total_male','total_female', 'random', 'first_trip_tz', 'country', 'Tour_ID'], axis =1)

  ## convert categorical columns to onehot values
  df = pd.get_dummies(df)

  ## rename columns to values acceptable by our models
  df = df.rename(columns= {'age_group_65+':'age_group_65_plus','age_group_<18':'age_group_18_less'})
  
  return df


def group_features(x, meta):
  if meta == 'activity':
    use = ['Conference Tourism','Widlife Tourism']
  elif meta == 'info':
    use = ['Tanzania Mission Abroad','Friends, relatives', 'Others']
  elif meta == 'purpose':
    use = ['Leisure and Holidays','Visiting Friends and Relatives']
  elif meta == 'travel_with':
    use = ['Alone']
  if x not in use:
    return 'others'
  else:
    return x

In [20]:
## create a copy of X and simplify
X_simple = X_.copy()
X_simple = simplify_features(X_simple)
## split data to train and test set
X_tn_sm, X_tt_sm, y_train, y_test = train_test_split(X_simple, y, test_size=0.2, random_state=42)

In [21]:
## finally fit model
model = xgb.XGBClassifier(seed=42, random_state = 42, max_depth = 5, average= 'weighted')
model.fit(X_tn_sm, y_train)
log_loss(y_test, model.predict_proba(X_tt_sm))

1.0819367189564089

## **THE END**